We are going to emulate the behavior of a magnetic field in microwires. Using the boundary element method and using Laplace as Green function, we're going to solve:

\begin{equation}\label{eq:Forma matricial ecuacion 1}
\begin{bmatrix}
\frac{1}{2}I+K_L^\Gamma & -V_L^\Gamma \\ 
\frac{1}{2}I-K_L^\Gamma & \frac{\varepsilon_1}{\varepsilon_2}V_L^\Gamma
\end{bmatrix}
\begin{bmatrix}
\varphi_{1d}\\ 
\frac{\partial}{\partial n}\varphi_{1d}
\end{bmatrix}
=\begin{bmatrix}
0\\ 
\frac{\varepsilon_2-\varepsilon_1}{\varepsilon_2} V_L^\Gamma \frac{\partial \varphi_{inc}}{\partial n}
\end{bmatrix}
\end{equation}

Being $K$ the double layer operator, $V$ the single layer operator and $I$ the identity. And using the subindex 1 for the interior of the studied item and the subindex 2 for the exterior of it. We begin by calling all the libraries that we'll use.

In [ ]:
#Import libraries
import bempp.api
import numpy as np
import timeit
import scipy as sp
import math
bempp.api.set_ipython_notebook_viewer()

In [ ]:
#Define grid
grid = bempp.api.import_grid("cilindro.msh")
matrix = bempp.api.import_grid("matriz2.msh")
number_of_elements = grid.leaf_view.entity_count(0)
number_of_elements2 = matrix.leaf_view.entity_count(0)
print("The grid has {0} elements.".format(number_of_elements))
print("The grid has {0} elements.".format(number_of_elements2))
grid.plot()
matrix.plot()

La unidad de la malla es del orden de $\mu m$, por lo que es necesario cambiar las unidades de ser necesario

In [ ]:
#Unidades de referencia
um = 1
mm = 1.e3
m = 1.e6

In [ ]:
#Data

#Frecuencia
omega = 1 / 3800

#Permeabilidad magnética
mu0 = 4. * np.pi * 1e-7 * 1.e6 * um  #micrometros

#Permitividad eléctrica
e0 = 1.7972083599999999+8.504766399999999e-09j
e1 = -2.9214+0.5895j *mu0
e2 = -1.3876520488233184+0.19220746083441781j  

#Propiedades del material
mu = (-2.9214 + 0.5895j) * mu0
e = (82629.2677-200138.2211*1.j) * e0
cc = 6.5e4 * 1.e-18  #micrometros #Conductividad eléctrica

#Permitividad eléctrica relativa del material c/r al medio
er = e1 / e0
er2 = e2 / e0
#Amplitud de la onda en micrometros
Amp = -1. * um #micrometros

#Longitud de onda
lamb = 3800
k = 2 * np.pi / lamb  

In [ ]:
#Set the Dirichlet and Neumann orders
order_neumann = 1
order_dirichlet = 1

NS = bempp.api.function_space(grid, "P", order_neumann)
DS = bempp.api.function_space(grid, "P", order_dirichlet)

NS2 = bempp.api.function_space(matrix, "P", order_neumann)
DS2 = bempp.api.function_space(matrix, "P", order_dirichlet)

#Print the degrees of freedom
print("BEM dofs: {0}".format(NS.global_dof_count))
print("BEM dofs: {0}".format(NS2.global_dof_count))
print("BEM dofs: {0}".format(DS.global_dof_count))
print("BEM dofs: {0}".format(DS2.global_dof_count))

In [ ]:
def stern_formulation(dirichl_space_in, neumann_space_in, dirichl_space_ex, neumann_space_ex, 
                      ep_in, ep_ex):

    # Functions to proyect the carges potential to the boundary with constants
    def green_func(x, n, domain_index, result):
        #result[:] = ((1 - er) / 1) * (Amp * n[2])
        result[:] = np.sum(1/np.linalg.norm( x - 1))/(4.*np.pi*ep_in)

    print("\nProjecting charges over surface...")
    charged_grid_fun  = bempp.api.GridFunction(dirichl_space_in, fun=green_func)

    rhs = np.concatenate([charged_grid_fun.coefficients, 
                          np.zeros(neumann_space_in.global_dof_count),
                          np.zeros(dirichl_space_ex.global_dof_count), 
                          np.zeros(neumann_space_ex.global_dof_count)])

    print("Defining operators...")
    # OPERATOR FOR INTERNAL SURFACE
    from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
    idn_in  = sparse.identity(dirichl_space_in, dirichl_space_in, dirichl_space_in)

    # Ec 1
    slp_in = laplace.single_layer(neumann_space_in, dirichl_space_in, dirichl_space_in)
    dlp_in = laplace.double_layer(dirichl_space_in, dirichl_space_in, dirichl_space_in)
    # Ec 2
    # adj_1T1 = laplace.single_layer(neumann_space_in, dirichl_space_in, dirichl_space_in)
    # adj_1T1 = laplace.double_layer(dirichl_space_in, dirichl_space_in, dirichl_space_in)

    slp_2T1 = laplace.single_layer(neumann_space_ex, dirichl_space_in, dirichl_space_in)
    dlp_2T1 = laplace.double_layer(dirichl_space_ex, dirichl_space_in, dirichl_space_in)

    # OPERATOR FOR EXTERNAL SURFACE
    idn_ex = sparse.identity(dirichl_space_ex, dirichl_space_ex, dirichl_space_ex)
    # Internal Boudary
    slp_1T2 = laplace.single_layer(neumann_space_in, dirichl_space_ex, dirichl_space_ex)
    # dlp_1T2 = laplace.double_layer(dirichl_space_in, dirichl_space_ex, dirichl_space_ex)

    slp_2T2 = laplace.single_layer(neumann_space_ex, dirichl_space_ex, dirichl_space_ex)
    dlp_2T2 = laplace.double_layer(dirichl_space_ex, dirichl_space_ex, dirichl_space_ex)
    # External Boundary
    slp_ex = modified_helmholtz.single_layer(neumann_space_ex, dirichl_space_ex, dirichl_space_ex, 1)
    dlp_ex = modified_helmholtz.double_layer(dirichl_space_ex, dirichl_space_ex, dirichl_space_ex, 1)

    ep = (ep_in/ep_ex)

    print("Creating operators...")
    # Matrix Assemble
    blocked = bempp.api.BlockedOperator(4, 4)
    blocked[0, 0] = .5 * idn_in + dlp_in
    blocked[0, 1] = -slp_in
    #blocked[0, 2] = 0
    #blocked[0, 3] = 0

    # Original formulation
    blocked[1, 0] = .5 * idn_in - dlp_in   # dlp_in = dlp_1T1
    blocked[1, 1] =  ep * slp_in           # slp_in = slp_1T1
    blocked[1, 2] =  dlp_2T1
    blocked[1, 3] = -slp_2T1

    #blocked[2, 0] = -dlp_1T2    ## eliminar**
    blocked[2, 1] =  ep * slp_1T2
    blocked[2, 2] = .5 * idn_ex + dlp_2T2
    blocked[2, 3] = -slp_2T2

    #blocked[3, 0] = 0
    #blocked[3, 1] = 0
    blocked[3, 2] = .5 * idn_ex - dlp_ex
    blocked[3, 3] = slp_ex
    A = blocked.strong_form()

    return A, rhs
stern_formulation(NS, DS, NS2, DS2, e0, e1)


In [ ]:
#import general_functions as gf, numpy as np, bempp.api, inspect, time, PBL, os, sys
dirichl_space_in = DS
neumann_space_in = NS
dirichl_space_ex = DS2
neumann_space_ex = NS2

print("Number of elements in  : {0}".format(dirichl_space_in.global_dof_count))
print("Number of elements out : {0}".format(dirichl_space_ex.global_dof_count))
print("Number of elements in  : {0}".format(neumann_space_in.global_dof_count))
print("Number of elements out : {0}".format(neumann_space_ex.global_dof_count))


def formulation(dirichl_space_in, neumann_space_in, dirichl_space_ex, neumann_space_ex, ep_in, ep_ex):

    # OPERATOR FOR INTERNAL SURFACE
    from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
        #Identidad Interior
    idn_in  = sparse.identity(dirichl_space_in, dirichl_space_in, dirichl_space_in)

        #Interior
    slp_in = laplace.single_layer(neumann_space_in, dirichl_space_in, dirichl_space_in)
    dlp_in = laplace.double_layer(dirichl_space_in, dirichl_space_in, dirichl_space_in)

    #Exterior a interior
    slp_2T1 = laplace.single_layer(neumann_space_ex, dirichl_space_in, dirichl_space_in)
    dlp_2T1 = laplace.double_layer(dirichl_space_ex, dirichl_space_in, dirichl_space_in)

    #Identidad Exterior
    idn_ex = sparse.identity(dirichl_space_ex, dirichl_space_ex, dirichl_space_ex)

    #Interior a Exterior
    slp_1T2 = laplace.single_layer(neumann_space_in, dirichl_space_ex, dirichl_space_ex)
    dlp_1T2 = laplace.double_layer(dirichl_space_in, dirichl_space_ex, dirichl_space_ex)

    #Exterior
    slp_2 = laplace.single_layer(neumann_space_ex, dirichl_space_ex, dirichl_space_ex)
    dlp_2 = laplace.double_layer(dirichl_space_ex, dirichl_space_ex, dirichl_space_ex)

    ep_in = e1
    ep_ex = e0
    ep = (ep_in/ep_ex)

    # Matrix Assemble
    blocked = bempp.api.BlockedOperator(4, 4)

    blocked[0, 0] = 0.5 * idn_in + dlp_in
    blocked[0, 1] = -slp_in
    #blocked[0, 2] = 0 * slp_2
    #blocked[0, 3] = 0 * slp_2

    blocked[1, 0] = 0.5 * idn_in - dlp_in   
    blocked[1, 1] =  ep * slp_in           
    blocked[1, 2] =  dlp_in
    blocked[1, 3] = - slp_in

    #blocked[2, 0] = - dlp_2    
    blocked[2, 1] =  ep * slp_2
    blocked[2, 2] = 0.5 * idn_ex + dlp_2
    blocked[2, 3] = - slp_2

    #blocked[3, 0] = 0 * slp_2
    #blocked[3, 1] = 0 * slp_2
    blocked[3, 2] = 0.5 * idn_ex - dlp_2
    blocked[3, 3] = slp_2

    A = blocked.strong_form()
    return A

A = formulation(DS, NS, DS2, NS2, e1, e0)
    
    # Functions
def funcion1(x, n, domain_index, result):
    result[:] = ((1 - er) / 1) * (Amp * n[2])

funcion_fun  = bempp.api.GridFunction(dirichl_space_in, fun=funcion1)

rhs = np.concatenate([funcion_fun.coefficients, 
        np.zeros(neumann_space_in.global_dof_count),
        np.zeros(dirichl_space_ex.global_dof_count), 
        np.zeros(neumann_space_ex.global_dof_count)])


In [ ]:
dirichl_space_in = DS
neumann_space_in = NS
dirichl_space_ex = DS2
neumann_space_ex = NS2

#def formulation(dirichl_space_in, neumann_space_in, dirichl_space_ex, neumann_space_ex, ep_in, ep_ex):

    # OPERATOR FOR INTERNAL SURFACE
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
    #Identidad Interior
idn_in  = sparse.identity(dirichl_space_in, dirichl_space_in, dirichl_space_in)

    #Interior
slp_in = laplace.single_layer(neumann_space_in, dirichl_space_in, dirichl_space_in)
dlp_in = laplace.double_layer(dirichl_space_in, dirichl_space_in, dirichl_space_in)

#Exterior a interior
slp_2T1 = laplace.single_layer(neumann_space_ex, dirichl_space_in, dirichl_space_in)
dlp_2T1 = laplace.double_layer(dirichl_space_ex, dirichl_space_in, dirichl_space_in)

#Identidad Exterior
idn_ex = sparse.identity(dirichl_space_ex, dirichl_space_ex, dirichl_space_ex)
    
#Interior a Exterior
slp_1T2 = laplace.single_layer(neumann_space_in, dirichl_space_ex, dirichl_space_ex)
dlp_1T2 = laplace.double_layer(dirichl_space_in, dirichl_space_ex, dirichl_space_ex)

#Exterior
slp_2 = laplace.single_layer(neumann_space_ex, dirichl_space_ex, dirichl_space_ex)
dlp_2 = laplace.double_layer(dirichl_space_ex, dirichl_space_ex, dirichl_space_ex)

ep_in = e1
ep_ex = e0
ep = (ep_in/ep_ex)

# Matrix Assemble
blocked = bempp.api.BlockedOperator(4, 4)
    
blocked[0, 0] = 0.5 * idn_in + dlp_in
blocked[0, 1] = -slp_in
blocked[0, 2] = 0 * slp_in
blocked[0, 3] = 0 * slp_in
    
blocked[1, 0] = 0.5 * idn_in - dlp_in   
blocked[1, 1] =  ep * slp_in           
blocked[1, 2] =  dlp_2T1
blocked[1, 3] = - slp_2T1

blocked[2, 0] = - dlp_1T2    
blocked[2, 1] =  ep * slp_1T2
blocked[2, 2] = 0.5 * idn_ex + dlp_2
blocked[2, 3] = - slp_2

blocked[3, 0] = 0 * slp_2
blocked[3, 1] = 0 * slp_2
blocked[3, 2] = 0.5 * idn_ex - dlp_2
blocked[3, 3] = slp_2
    
A = blocked.strong_form()
#    return A

#A = formulation(DS, NS, DS2, NS2, e1, e0)
    
    # Functions
def funcion1(x, n, domain_index, result):
    result[:] = ((1 - er) / 1) * (Amp * n[2])

funcion_fun  = bempp.api.GridFunction(dirichl_space_in, fun=funcion1)

rhs = np.concatenate([funcion_fun.coefficients, 
        np.zeros(neumann_space_in.global_dof_count),
        np.zeros(dirichl_space_ex.global_dof_count), 
        np.zeros(neumann_space_ex.global_dof_count)])


In [ ]:
#Dirichlet Segment
slp = bempp.api.operators.boundary.laplace.single_layer(NS,DS,DS)
dlp = bempp.api.operators.boundary.laplace.double_layer(DS,DS,DS)
id = bempp.api.operators.boundary.sparse.identity(DS,DS,DS)

slp2 = bempp.api.operators.boundary.laplace.single_layer(NS2,DS2,DS2)
dlp2 = bempp.api.operators.boundary.laplace.double_layer(DS2,DS2,DS2)
id2 = bempp.api.operators.boundary.sparse.identity(DS2,DS2,DS2)

slp21 = bempp.api.operators.boundary.laplace.single_layer(NS2,DS,DS)
dlp21 = bempp.api.operators.boundary.laplace.double_layer(DS2,DS,DS)
id21 = bempp.api.operators.boundary.sparse.identity(DS2,DS,DS)

slp12 = bempp.api.operators.boundary.laplace.single_layer(NS,DS2,DS2)
dlp12 = bempp.api.operators.boundary.laplace.double_layer(DS,DS2,DS2)
id12 = bempp.api.operators.boundary.sparse.identity(DS,DS2,DS2)

\begin{equation}\label{eq:Forma matricial ecuacion 1}
\begin{bmatrix}
\frac{1}{2}I+K & -V \\ 
\frac{1}{2}I-K & \frac{\varepsilon_1}{\varepsilon_2}V
\end{bmatrix}
\end{equation}

In [ ]:
blocked = bempp.api.BlockedOperator(4, 4)

blocked[0][0] = (0.5 * id + dlp).weak_form()
blocked[0][1] = - slp.weak_form()
#blocked[0][2] = 0*slp.weak_form()
#blocked[0][3] = 0*slp.weak_form()
blocked[1][0] = (0.5 * id - dlp).weak_form()
blocked[1][1] = (er * slp).weak_form()
blocked[1][2] = dlp12.weak_form()
blocked[1][3] = - er2 * slp12.weak_form()
blocked[2][0] = - dlp21.weak_form()
blocked[2][1] = (er * slp21).weak_form()
blocked[2][2] = (0.5 * id + dlp2).weak_form()
blocked[2][3] = - er2 * slp2.weak_form()
#blocked[3][0] = 0*slp.weak_form()
#blocked[3][1] = 0*slp.weak_form()
blocked[3][2] = (0.5 * id + dlp2).weak_form() 
blocked[3][3] = - slp2.weak_form()

#A = blocked.strong_form()

\begin{equation}\label{eq:Forma matricial ecuacion 1}
\begin{bmatrix}
0\\ 
\frac{\varepsilon_2-\varepsilon_1}{\varepsilon_2}V_L^\Gamma \frac{\partial \varphi_{inc}}{\partial n}
\end{bmatrix}
\end{equation}

In [ ]:
print(funcion_fun)

In [ ]:
#Definition of functions
def funcion1(x, n, domain_index, result):
    result[:] = ((1 - er) / 1) * (Amp * n[2])
    
#Functions in the function space    
funcion_fun = bempp.api.GridFunction(DS, fun=funcion1)
funcion_fun = slp * funcion_fun

In [ ]:
# The rhs from the zeros
rhs_fem = np.zeros(number_of_elements)
# The rhs from the function
rhs_bem = funcion_fun.projections(DS)
# The combined rhs
rhs = np.concatenate([rhs_fem, rhs_fem, rhs_fem, rhs_bem])
print(rhs)

In [ ]:
start = timeit.default_timer()
#Solving the matrix system
from scipy.sparse.linalg import gmres

#it_count = 0
#def count_iterations(x):
#    global it_count
#    it_count += 1

#soln, info = gmres(blocked, rhs, callback=count_iterations)
print(rhs)
print(blocked)
sol, info, it_count = bempp.api.linalg.gmres(blocked, rhs,
                                            use_strong_form=True, return_iteration_count=True, tol=1e-3)

print("The linear system was solved in {0} iterations".format(it_count))

$$\begin{bmatrix}
\varphi_{1d}\\ 
\frac{\partial}{\partial n}\varphi_{1d}
\end{bmatrix}$$

In [ ]:
#Divide the solution
solution_dirichl=soln[:number_of_elements]
solution_neumann=soln[number_of_elements:]

print(soln)

In [ ]:
vertices = grid.leaf_view.vertices
elements = grid.leaf_view.elements

In [ ]:
elements = list(grid.leaf_view.entity_iterator(0))

$$p_i=\epsilon_2\displaystyle\sum^n_{i=0}\left[r_i\frac{\partial}{\partial n_j}\phi_{2s} d\Gamma-n_i\phi d\Gamma\right]$$

In [ ]:
suma = 0
suma1 = 0
suma2 = 0
for i in range(number_of_elements):
    corners = elements[i].geometry.corners
    area = elements[i].geometry.volume
    
    p1 = corners[0]
    p2 = corners[1]
    p3 = corners[2]

    center = (p1 + p2 + p3) / 3

    v1 = p2 - center
    v2 = p3 - center

    normalv = np.cross(v2, v1)
    a, b, c = normalv
    norm_normalv = np.linalg.norm(normalv)
    
    normal = normalv/norm_normalv
    phi_inc = solution_dirichl[i]
    dphi_dn_inc = solution_neumann[i]
    
    suma = suma + (center[0] * dphi_dn_inc - normal[0] * phi_inc) * area
    suma1 = suma1 + (center[1] * dphi_dn_inc - normal[1] * phi_inc) * area
    suma2 = suma2 + (center[2] * dphi_dn_inc - normal[2] * phi_inc) * area
    
p_i=[e0 * suma, e0 * suma1, e0 * suma2]
print(p_i)

$$ E_{2s} = \frac{1}{4\pi\epsilon_2}k^2 \frac{e^{ikr}}{r}(\hat{r}\times p )\times \hat{r}$$

$$E_{2s}(r)_{r\to\infty}=\frac{e^{ikr}}{r}F(k,k_0)$$

$$C_{ext}=\frac{4\pi}{k}Im\left[\frac{\hat{e}_i}{|E_i|}F(k=k_0,k_0)\right]$$

In [ ]:
r1 = (1,0,0)
r2 = (0,0,1)
aux1 = np.cross(r1,p_i)
aux2 = np.cross(aux1,r1)
C1 = np.dot(r2,aux2) * k**2 / (e0 * Amp) 
C_ext = (number_of_elements / k.real * C1.imag * 0.01)

print(C_ext)